In [3]:
import os
import gc
import pickle
import anndata
import scanpy as sc
import pandas as pd
import numpy as np
from scipy import sparse
from scipy.stats import spearmanr, pearsonr
from scipy.spatial import distance_matrix
import matplotlib.pyplot as plt

import spatialdm as sd

Create a smaller set of the scanpy set

In [4]:
# Check the number of cells
path_chuvio_L1_1 = "/work/PRTNR/CHUV/DIR/rgottar1/spatial/env/astear/data/rcd_annot/chuvio_L1_1_RCTD_lvl3.h5ad"
chuvio_L1_1 = sc.read_h5ad(path_chuvio_L1_1)

chuvio_L1_1.obsm['spatial'] = chuvio_L1_1.obs[['x_centroid', 'y_centroid']].to_numpy()

In [5]:
chuvio_L1_1

AnnData object with n_obs × n_vars = 71187 × 313
    obs: 'orig.ident', 'nCount_originalexp', 'nFeature_originalexp', 'cell_id', 'x_centroid', 'y_centroid', 'transcript_counts', 'control_probe_counts', 'control_codeword_counts', 'unassigned_codeword_counts', 'deprecated_codeword_counts', 'total_counts', 'cell_area', 'nucleus_area', 'sample_id', 'sum', 'detected', 'subsets_Mito_sum', 'subsets_Mito_detected', 'subsets_Mito_percent', 'total', 'libsize_drop', 'sizeFactor', 'nCount_SCT', 'nFeature_SCT', 'SCT_snn_res.0.8', 'seurat_clusters', 'singler_annotation', 'ident', 'RCTD_lvl3'
    uns: 'X_name'
    obsm: 'PCA', 'TSNE', 'UMAP', 'spatial'
    layers: 'logcounts'

Use scanpy.pp.subsample to create and save a subsample

In [9]:
L1_1_sub20K = sc.pp.subsample(chuvio_L1_1, n_obs=20000)

In [13]:
L1_1_sub20K = chuvio_L1_1

In [14]:
file_path = "/work/PRTNR/CHUV/DIR/rgottar1/spatial/env/astear/Programming/synthetic_data/subsets/L1_1_sub20K.h5ad"

# Save the AnnData object to the specified path
L1_1_sub20K.write(file_path)

## Gene sub sample

In [ ]:
List of genes we are interested in : 
CCL
CCL2
CCL3
CCL4
CCL5
CCL17
CCL19
CCL22
CCR1
CCR2
CCR2
CCR3
CCR4
CCR5
CCR7
EGF
EGFR
EREG
ERBB2
BANK1
CXCR3
DES
HGF 
IL32 
RNASE1 
XCL1
LUM
STAT4
TRDV2

not interested random : 
BANK1
CXCR3
DES
HGF 
IL32 
RNASE1 
XCL1
LUM
STAT4
TRDV2

In [10]:
chuvio_L1_1

AnnData object with n_obs × n_vars = 71187 × 313
    obs: 'orig.ident', 'nCount_originalexp', 'nFeature_originalexp', 'cell_id', 'x_centroid', 'y_centroid', 'transcript_counts', 'control_probe_counts', 'control_codeword_counts', 'unassigned_codeword_counts', 'deprecated_codeword_counts', 'total_counts', 'cell_area', 'nucleus_area', 'sample_id', 'sum', 'detected', 'subsets_Mito_sum', 'subsets_Mito_detected', 'subsets_Mito_percent', 'total', 'libsize_drop', 'sizeFactor', 'nCount_SCT', 'nFeature_SCT', 'SCT_snn_res.0.8', 'seurat_clusters', 'singler_annotation', 'ident', 'RCTD_lvl3'
    uns: 'X_name'
    obsm: 'PCA', 'TSNE', 'UMAP', 'spatial'
    layers: 'logcounts'

In [12]:
#chuvio_L1_1.var_names.tolist()

In [13]:
genes_to_keep = [
    'CCL', 'CCL2', 'CCL3', 'CCL4', 'CCL5', 'CCL17', 'CCL19', 'CCL22',
    'CCR1', 'CCR2', 'CCR3', 'CCR4', 'CCR5', 'CCR7',
    'EGF', 'EGFR', 'EREG', 'ERBB2',
    'BANK1', 'CXCR3', 'DES', 'HGF', 'IL32', 'RNASE1', 'XCL1',
    'LUM', 'STAT4', 'TRDV2'
]


In [14]:
mask = chuvio_L1_1.var_names.isin(genes_to_keep)

# Create a new AnnData object with only the selected genes
chuvio_L1_1_20genes = chuvio_L1_1[:, mask]

In [17]:
save_dir = '/work/PRTNR/CHUV/DIR/rgottar1/spatial/env/astear/Programming/synthetic_data/20genes'
filename = 'chuvio_L1_1_20genes.h5ad'
full_path = os.path.join(save_dir, filename)

# Ensure the directory exists
os.makedirs(save_dir, exist_ok=True)

# Save the new AnnData object
chuvio_L1_1_20genes.write_h5ad(full_path)

In [18]:
path_chuvio_L1_1 = "/work/PRTNR/CHUV/DIR/rgottar1/spatial/Owkin_Pilot_Intermediate/Visium_qcd_anndata/B1_2_qcd.h5ad"
chuvio_L1_1 = sc.read_h5ad(path_chuvio_L1_1)

In [20]:
chuvio_L1_1.obs

,in_tissue,array_row,array_col,sample_id,sum,detected,subsets_Mito_sum,subsets_Mito_detected,subsets_Mito_percent,total,mito_drop,libsize_drop,patho_exclude,edge,all_drop,Region,Barcode,Level4_decon_max,Level4_decon_mixtypes
AACACTTGGCAAGGAA-1,True,47,71,sample01,13437.0,4522,187.0,11,1.391680,13437.0,False,False,False,False,False,Tumor_Stroma_mix,AACACTTGGCAAGGAA-1,Fibroblast,Fibroblast
AACAGGATTCATAGTT-1,True,49,43,sample01,15083.0,5439,207.0,11,1.372406,15083.0,False,False,False,False,False,Intratumoral_Stroma,AACAGGATTCATAGTT-1,Tu_B1_MUCL1_necrosis,Fibroblast - Tu_B1_MUCL1_necrosis
AACATACTCATATGCG-1,True,50,6,sample01,23347.0,6315,257.0,11,1.100784,23347.0,False,False,False,False,False,Immune_Cell_mix,AACATACTCATATGCG-1,Fibroblast,Fibroblast - T_CTL
AACATAGTCTATCTAC-1,True,58,26,sample01,16458.0,5943,381.0,11,2.314984,16458.0,False,False,False,False,False,Intratumoral_Stroma,AACATAGTCTATCTAC-1,Tu_B1_MUCL1_necrosis,Fibroblast - Tu_B1_MUCL1_necrosis
AACCAAGGTATCAGGC-1,True,38,104,sample01,21199.0,6954,411.0,11,1.938771,21199.0,False,False,False,False,False,Tumor_Stroma_mix,AACCAAGGTATCAGGC-1,Tu_B1_MUCL1_necrosis,Fibroblast - Tu_B1_MUCL1_necrosis
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
TGTTCATTGGTCCAAG-1,True,55,99,sample01,7149.0,4066,170.0,11,2.377955,7149.0,False,False,False,False,False,Breast_Carcinoma_in_situ,TGTTCATTGGTCCAAG-1,Fibroblast,Fibroblast - Tu_B1_MUCL1_necrosis
TGTTCCTCACATTAAT-1,True,36,100,sample01,21075.0,6732,422.0,11,2.002372,21075.0,False,False,False,False,False,Tumor_Stroma_mix,TGTTCCTCACATTAAT-1,Tu_B1_MUCL1_necrosis,Tu_B1_MUCL1_necrosis
TGTTCTGCTCTGTCGT-1,True,39,39,sample01,9951.0,3974,102.0,11,1.025023,9951.0,False,False,False,False,False,Intratumoral_Stroma,TGTTCTGCTCTGTCGT-1,Fibroblast,Fibroblast
TGTTGCGTCCAAGATT-1,True,51,65,sample01,11638.0,4676,146.0,10,1.254511,11638.0,False,False,False,False,False,Intratumoral_Stroma,TGTTGCGTCCAAGATT-1,Fibroblast,Fibroblast
